In [ ]:
import pandas as pd
import os

import sys
sys.path.append("..")
from src.Eval import aggregated_results_llm, results_to_dataframe, silent_eval_llm, get_iobs_from_data, parse_filename
from src.Utils import parse_pred_file, parse_preds, read_jsonlines, read_IOB_file

results_postcut = aggregated_results_llm("../output/reddit+shsyt_cloze/post_cutoff")
results_seen = aggregated_results_llm("../output/reddit+shsyt_cloze/seen")
results_unseen = aggregated_results_llm("../output/reddit+shsyt_cloze/unseen")


In [ ]:
results_to_dataframe(results_postcut)


In [ ]:
silent_eval_llm("../output/reddit+shsyt_cloze/post_cutoff/firefunction-v2/dataset1_0shot.jsonl")


In [ ]:
path="../output/reddit+shsyt_cloze/post_cutoff/firefunction-v2/dataset1_35shot_tfidf.jsonl"
parse_preds(path)


In [ ]:
read_jsonlines(path)


In [ ]:
results_to_dataframe(results_seen)


In [ ]:
results_to_dataframe(results_unseen)


# Analysis

### Parse preds

In [100]:
model = "firefunction-v2"
data_path = f"../data/dataset/reddit+shsyt_cloze/{model}/"
output_path = f"../output/reddit+shsyt_cloze"
group_paths = os.listdir(data_path)

results = {}

for group in group_paths:
    group_path = os.path.join(data_path, group)
    
    titles = []
    performers = []
    texts = []
    tags = []
    pred_tags = {}

    for dataset in os.listdir(group_path):
        _texts, _tags = read_IOB_file(os.path.join(group_path, dataset, "test.IOB"))
        texts += _texts
        tags += _tags

        _output_path = os.path.join(output_path, group, model)
        for pred_file in [f for f in os.listdir(_output_path) if dataset in f]:
            dataset, k, sampling = parse_filename(pred_file)
            pred_path = os.path.join(_output_path, pred_file)
            pred_data = read_jsonlines(pred_path)
            true_iobs, pred_iobs = get_iobs_from_data(pred_data)
            key = str(k) + sampling

            if pred_tags.get(key):
                pred_tags[key] += pred_iobs
            else:
                pred_tags[key] = pred_iobs
        
        titles += [d.get("titles") for d in pred_data]
        performers += [d.get("performers") for d in pred_data]

    results['_'.join((group, "titles"))] = titles
    results['_'.join((group, "performers"))] = performers
    results['_'.join((group, "TEXT"))] = [list(a) for a in texts]
    results['_'.join((group, "IOB"))] = [list(a) for a in tags]

    for key in pred_tags.keys():
        results['_'.join((group, "IOB-" + key))] = pred_tags[key]

data = pd.concat(
    [pd.read_parquet("../data/dataset/reddit+shsyt_cloze/data_template.parquet").reset_index(),
     pd.DataFrame(results)],
     axis=1
)


In [101]:
data


,index,id,set_id,Origin,TEXT,IOB,subset,yt_id,has_WoA,has_Artist,...,post_cutoff_IOB-35rand,post_cutoff_IOB-0rand,post_cutoff_IOB-35tfidf,unseen_titles,unseen_performers,unseen_TEXT,unseen_IOB,unseen_IOB-35rand,unseen_IOB-0rand,unseen_IOB-35tfidf
0,46,21_L1-t3YvFH40,21.0,YouTube,"[gladys, knight, &, the, pips, -, yesterday]","[B-Artist, I-Artist, O, B-Artist, I-Artist, O,...",4,L1-t3YvFH40,True,True,...,"[B-Artist, O, B-Artist, O, O, O, O]","[O, O, O, O, O, O, O]","[B-Artist, O, B-Artist, O, O, O, O]",[the things you said],"[ray charles, one hit wonder [ 1 ]]","[ray, charles, &, one, hit, wonder, [, 1, ], -...","[B-Artist, I-Artist, O, B-Artist, I-Artist, I-...","[B-Artist, I-Artist, O, B-WoA, I-WoA, I-WoA, O...","[B-Artist, I-Artist, O, O, O, O, O, O, O, O, B...","[B-Artist, I-Artist, O, B-WoA, I-WoA, I-WoA, O..."
1,79,21_4SE8UvqaSWQ,21.0,YouTube,"[hooked, on, the, beatles, -, the, end]","[B-WoA, I-WoA, I-WoA, I-WoA, O, B-WoA, I-WoA]",4,4SE8UvqaSWQ,True,False,...,"[B-WoA, B-WoA, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O]","[B-Artist, B-WoA, O, O, O, O, O, O, O, O, O, O]","[you got to reap what you sow, moments like this]",[],"[you, got, to, reap, what, you, sow, -, moment...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-W...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-W...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-W...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-W..."
2,170,58_hQz-aZH-ozU,58.0,YouTube,"[juan, rozoff, -, plaisir, d, amour]","[B-Artist, I-Artist, O, B-WoA, I-WoA, I-WoA]",1,hQz-aZH-ozU,True,True,...,"[O, O, O]","[O, O, O]","[B-Artist, O, B-WoA]",[walk on by],[leroy van dyke],"[leroy, van, dyke, -, walk, on, by]","[B-Artist, I-Artist, I-Artist, O, B-WoA, I-WoA...","[B-Artist, I-Artist, I-Artist, O, B-WoA, I-WoA...","[B-Artist, I-Artist, I-Artist, O, B-WoA, I-WoA...","[B-Artist, I-Artist, I-Artist, O, B-WoA, I-WoA..."
3,249,58_-cbPbDQhoyg,58.0,YouTube,"[i, cant, help, falling, in, love, -, enrico, ...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, O, ...",1,-cbPbDQhoyg,True,True,...,"[B-WoA, I-WoA, O, O, O, O, B-Artist, I-Artist,...","[O, O, O, O, O, O, B-Artist, I-Artist, I-Artist]","[B-WoA, I-WoA, O, O, O, O, B-Artist, I-Artist, O]",[ridin high],[ella fitzgerald],"[ridin, high, -, ella, fitzgerald]","[B-WoA, I-WoA, O, B-Artist, I-Artist]","[B-WoA, I-WoA, O, B-Artist, I-Artist]","[B-WoA, I-WoA, O, B-Artist, I-Artist]","[B-WoA, I-WoA, O, B-Artist, I-Artist]"
4,367,81_kRssJvqEfzY,81.0,YouTube,"[spanish, harlem, -, alan, haven, and, tony, c...","[O, O, O, B-Artist, I-Artist, O, B-Artist, I-A...",1,kRssJvqEfzY,True,True,...,"[B-WoA, I-WoA, O, B-WoA, O, B-Artist, I-Artist...","[B-WoA, I-WoA, O, B-WoA, O, B-WoA, I-WoA, O, B...","[B-Artist, I-Artist, O, B-WoA, O, B-Artist, I-...","[hor mein lied , violetta]","[gunter - kallmann - chor, orchester]","[spanish, harlem, -, gunter, -, kallmann, -, c...","[O, O, O, B-Artist, I-Artist, I-Artist, I-Arti...","[B-WoA, I-WoA, O, B-Artist, O, B-Artist, I-Art...","[B-WoA, I-WoA, O, B-Artist, O, B-Artist, O, O,...","[B-WoA, I-WoA, O, B-Artist, O, O, O, O, O, O, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,2668,dataset4_739,12661.0,Reddit,"[looking, for, songs, similar, to, florida, ki...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...",2,739,True,True,...,"[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...","[O, O, O, O, O, B-WoA, I-WoA, O, B-WoA, I-WoA, O]","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...",[philadelphia lawyer],[bonnie owens],"[looking, for, songs, similar, to, philadelphi...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A..."
1054,2672,dataset4_743,12665.0,Reddit,"[gently, wind, |, joshua, speedy]","[B-WoA, I-WoA, O, B-Artist, I-Artist]",1,743,True,True,...,"[B-WoA, I-WoA, I-WoA, O, B-Artist, I-Artist, I...","[B-WoA, I

,index,id,set_id,Origin,TEXT,IOB,subset,yt_id,has_WoA,has_Artist,...,post_cutoff_IOB-35rand,post_cutoff_IOB-0rand,post_cutoff_IOB-35tfidf,unseen_titles,unseen_performers,unseen_TEXT,unseen_IOB,unseen_IOB-35rand,unseen_IOB-0rand,unseen_IOB-35tfidf
0,46,21_L1-t3YvFH40,21.0,YouTube,"[gladys, knight, &, the, pips, -, yesterday]","[B-Artist, I-Artist, O, B-Artist, I-Artist, O,...",4,L1-t3YvFH40,True,True,...,"[B-Artist, O, B-Artist, O, O, O, O]","[O, O, O, O, O, O, O]","[B-Artist, O, B-Artist, O, O, O, O]",[the things you said],"[ray charles, one hit wonder [ 1 ]]","[ray, charles, &, one, hit, wonder, [, 1, ], -...","[B-Artist, I-Artist, O, B-Artist, I-Artist, I-...","[B-Artist, I-Artist, O, B-WoA, I-WoA, I-WoA, O...","[B-Artist, I-Artist, O, O, O, O, O, O, O, O, B...","[B-Artist, I-Artist, O, B-WoA, I-WoA, I-WoA, O..."
1,79,21_4SE8UvqaSWQ,21.0,YouTube,"[hooked, on, the, beatles, -, the, end]","[B-WoA, I-WoA, I-WoA, I-WoA, O, B-WoA, I-WoA]",4,4SE8UvqaSWQ,True,False,...,"[B-WoA, B-WoA, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O]","[B-Artist, B-WoA, O, O, O, O, O, O, O, O, O, O]","[you got to reap what you sow, moments like this]",[],"[you, got, to, reap, what, you, sow, -, moment...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-W...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-W...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-W...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-W..."
2,170,58_hQz-aZH-ozU,58.0,YouTube,"[juan, rozoff, -, plaisir, d, amour]","[B-Artist, I-Artist, O, B-WoA, I-WoA, I-WoA]",1,hQz-aZH-ozU,True,True,...,"[O, O, O]","[O, O, O]","[B-Artist, O, B-WoA]",[walk on by],[leroy van dyke],"[leroy, van, dyke, -, walk, on, by]","[B-Artist, I-Artist, I-Artist, O, B-WoA, I-WoA...","[B-Artist, I-Artist, I-Artist, O, B-WoA, I-WoA...","[B-Artist, I-Artist, I-Artist, O, B-WoA, I-WoA...","[B-Artist, I-Artist, I-Artist, O, B-WoA, I-WoA..."
3,249,58_-cbPbDQhoyg,58.0,YouTube,"[i, cant, help, falling, in, love, -, enrico, ...","[B-WoA, I-WoA, I-WoA, I-WoA, I-WoA, I-WoA, O, ...",1,-cbPbDQhoyg,True,True,...,"[B-WoA, I-WoA, O, O, O, O, B-Artist, I-Artist,...","[O, O, O, O, O, O, B-Artist, I-Artist, I-Artist]","[B-WoA, I-WoA, O, O, O, O, B-Artist, I-Artist, O]",[ridin high],[ella fitzgerald],"[ridin, high, -, ella, fitzgerald]","[B-WoA, I-WoA, O, B-Artist, I-Artist]","[B-WoA, I-WoA, O, B-Artist, I-Artist]","[B-WoA, I-WoA, O, B-Artist, I-Artist]","[B-WoA, I-WoA, O, B-Artist, I-Artist]"
4,367,81_kRssJvqEfzY,81.0,YouTube,"[spanish, harlem, -, alan, haven, and, tony, c...","[O, O, O, B-Artist, I-Artist, O, B-Artist, I-A...",1,kRssJvqEfzY,True,True,...,"[B-WoA, I-WoA, O, B-WoA, O, B-Artist, I-Artist...","[B-WoA, I-WoA, O, B-WoA, O, B-WoA, I-WoA, O, B...","[B-Artist, I-Artist, O, B-WoA, O, B-Artist, I-...","[hor mein lied , violetta]","[gunter - kallmann - chor, orchester]","[spanish, harlem, -, gunter, -, kallmann, -, c...","[O, O, O, B-Artist, I-Artist, I-Artist, I-Arti...","[B-WoA, I-WoA, O, B-Artist, O, B-Artist, I-Art...","[B-WoA, I-WoA, O, B-Artist, O, B-Artist, O, O,...","[B-WoA, I-WoA, O, B-Artist, O, O, O, O, O, O, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,2668,dataset4_739,12661.0,Reddit,"[looking, for, songs, similar, to, florida, ki...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...",2,739,True,True,...,"[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...","[O, O, O, O, O, B-WoA, I-WoA, O, B-WoA, I-WoA, O]","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...",[philadelphia lawyer],[bonnie owens],"[looking, for, songs, similar, to, philadelphi...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A...","[O, O, O, O, O, B-WoA, I-WoA, O, B-Artist, I-A..."
1054,2672,dataset4_743,12665.0,Reddit,"[gently, wind, |, joshua, speedy]","[B-WoA, I-WoA, O, B-Artist, I-Artist]",1,743,True,True,...,"[B-WoA, I-WoA, I-WoA, O, B-Artist, I-Artist, I...","[B-WoA, I

In [97]:
pd.read_parquet("../data/dataset/reddit+shsyt_cloze/data_template.parquet").count()

id               1058
set_id           1058
Origin           1058
TEXT             1058
IOB              1058
subset           1058
yt_id            1058
has_WoA          1058
has_Artist       1058
IOB_template     1058
TEXT_template    1058
O_tags           1058
nO_tags          1058
dtype: int64

In [95]:
data.count()

seen_titles                1058
seen_performers            1058
seen_TEXT                  1058
seen_IOB                   1058
seen_IOB-35rand            1058
seen_IOB-0rand             1058
seen_IOB-35tfidf           1058
post_cutoff_titles         1058
post_cutoff_performers     1058
post_cutoff_TEXT           1058
post_cutoff_IOB            1058
post_cutoff_IOB-35rand     1058
post_cutoff_IOB-0rand      1058
post_cutoff_IOB-35tfidf    1058
unseen_titles              1058
unseen_performers          1058
unseen_TEXT                1058
unseen_IOB                 1058
unseen_IOB-35rand          1058
unseen_IOB-0rand           1058
unseen_IOB-35tfidf         1058
dtype: int64